# 01 Random agent baseline

#### 👉Bir Pekiştirmeli Öğrenme problemini çözmeye çalışmadan önce, onun zorluğunu anlamalıyız.

#### 👉 Bunu yapmak için, görevi çok fazla kafa yormadan gerçekleştirebilecek ve performansını değerlendirebileceğimiz kukla bir ajan tasarlamamız gerekir.

#### 👉Bunu yapmanın basit bir yolu, ortamın mevcut durumuna dikkat etmeden bir sonraki eylemini rastgele seçen bir Rastgele Ajan (Random Agent) kullanmaktır.

**Temel model olarak rastgele bir ajan kullanalım.**

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline
%config InlineBackend.figure_format = 'svg'

## Çevre (Environment) 🌎

In [ ]:
import gym
env = gym.make("Taxi-v3").env

## Rastgele Ajan (Random agent) 🤖🍷

In [ ]:
class RandomAgent:
    """
    This taxi driver selects actions randomly.
    You better not get into this taxi!
    """
    def __init__(self, env):
        self.env = env

    def get_action(self, state) -> int:
        """
        We have `state` as an input to keep
        a consistent API for all our agents, but it
        is not used.
        
        i.e. The agent does not consider the state of
        the environment when deciding what to do next.
        This is why we call it "random".
        """
        return self.env.action_space.sample()

agent = RandomAgent(env)

## Bu sürücüyü sabit bir durumdan (`state = 123`) başlayarak değerlendirelim

In [ ]:
# set initial state of the environment
env.reset()
state = 123
env.s = state

epochs = 0
penalties = 0  # wrong pick up or dropp off
reward = 0

# store frames to latter plot them
frames = []

done = False

while not done:
    
    action = agent.get_action(state)
    
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1} of {len(frames)}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.01)
        
print_frames(frames)

#### Oldukça kötü bir sürüş, değil mi?

## Performansı ölçmek için histogramlar oluşturalım

Daha temsili bir performans ölçüsü elde etmek için, her seferinde rastgele bir durumda başlayarak aynı değerlendirme döngüsünü n=100 kez tekrarlayabiliriz.

In [ ]:
from tqdm import tqdm

n_episodes = 100

# For plotting metrics
timesteps_per_episode = []
penalties_per_episode = []

for i in tqdm(range(0, n_episodes)):
    
    # reset environment to a random state
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        
        action = agent.get_action(state)       
        next_state, reward, done, info = env.step(action) 
               
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
    
    timesteps_per_episode.append(epochs)
    penalties_per_episode.append(penalties)

### **timesteps_per_episode ve notice_per_episode'u çizerseniz**, aracı daha fazla bölüm tamamladıkça bunların hiçbirinin azalmadığını gözlemleyebilirsiniz. Başka bir deyişle, **temsilci hiçbir şey ÖĞRENMİYOR.**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (12, 4))
ax.set_title("Timesteps to complete ride")    
pd.Series(timesteps_per_episode).plot(kind="line")
plt.show()

fig, ax = plt.subplots(figsize = (12, 4))
ax.set_title("Penalties per ride")    
pd.Series(penalties_per_episode).plot(kind="line")
plt.show()

### Performansın özet istatistiklerini istiyorsanız, ortalamaları alabilirsiniz:

In [ ]:
print(f'Avg steps to complete ride: {np.array(timesteps_per_episode).mean()}')
print(f'Avg penalties to complete ride: {np.array(penalties_per_episode).mean()}')